# Gibbs Sampling

## 1. Initialize $\Theta^{(0)}$, $\pi^{(0)}$,$\kappa^{(0)}$.

In [1]:
# Desc:
#   Initalizes the parameters.
#   Assume theta0 follows an uniform distribution theta0 ~ beta(1,1)
#   Assume kappa0 follows an uniform distribution kappa0 ~ dirichlet(1)
# Input: 
#   data: a N * P matrix
#   C: the number of category.
# Output:
#   list(theta0, pi0, kappa0)
#   beta0: prior of theta, c(a, b)
#   pi0: initial pi
#   kappa0: initial kappa

gibbs_init = function(data, C) {
  beta0  = c(1,1)
  pi0 = rep(1/C, C)
  kappa0 = sample(1:C, dim(data)[1], replace = TRUE)
  alpha0 = rep(1,C)
  return(list(beta0 = beta0, pi0 = pi0, kappa0 = kappa0, alpha0 = alpha0))
}

In [2]:
# test data
data = matrix(c(1,0,1,0,0,1,1,0,0,1,1,1,1,0,0,1,1,0,0,0), nrow=5, ncol=4, byrow = TRUE)
C = 2
gibbs_init(data, C)

$beta0
[1] 1 1

$pi0
[1] 0.5 0.5

$kappa0
[1] 2 2 2 1 1

$alpha0
[1] 1 1

## 2. Sample $\Theta^{(t)}| X, \kappa^{(t-1)}$ 

In [3]:
# Desc:
#   Samples the theta parameters using a beta distrobution
# Input:
#   data: a N * P matrix
#   beta0: prior of theta, c(a, b)
#   kappa0: current kappa
# Output:
#   theta: updated theta, a C * P matrix

sample_theta = function(data, beta0, kappa0, C) {
  theta1 = matrix(rep(0,C*dim(data)[2]), nrow=C, ncol=dim(data)[2])
  for (c in 1:C) {
    data_c = data[which(kappa0 == c),]
    n_c = dim(data_c)[1]
    # Handle the case where data_c is a vector
    if(is.null(n_c)){
      n_c = length(data_c)
      n_cj = data_c
    }
    else{
      n_cj = colSums(data_c)
    }
    for (j in 1:length(n_cj)) {
      n_cj[j] = rbeta(1, beta0[1]+n_cj[j], beta0[2]+n_c-n_cj[j])
    }
    theta1[c,] = n_cj
  }
  return(theta1)
}

In [4]:
# test data
data = matrix(c(1,0,1,0,0,1,1,0,0,1,1,1,1,0,0,1,1,0,0,0), nrow=5, ncol=4, byrow = TRUE)
theta0 = c(1,1)
kappa0 = c(1,2,1,1,2)
C = 2
sample_theta(data, theta0, kappa0, C)

0.7571081,0.2896043,0.8341133,0.41821373
0.4431751,0.4149270,0.2297970,0.05560299


## 3. Sample $\kappa^{(t)}| X,\Theta^{(t)},\pi^{(t-1)}$

In [5]:
# Desc:
#   Samples the category assignments using a categorical distrobution
# Input:
#   data: a N * P matrix
#   theta: current theta, a C * P matrix
#   C: number of category
#   pi0: current pi
# Output:
#   kappa1: updated kappa, a vector of length N

sample_kappa = function(data, theta, C, pi0) {
  kappa1 = rep(NA, dim(data)[1])
  for (i in 1:dim(data)[1]) {
    prob = rep(NA, C)
    for (c in 1:C) {
      prob[c] = prod((theta[c,]^data[i,])*((1 - theta[c,])^(1 - data[i,]))) * pi0[c]
    }
    prob = prob / sum(prob)
    kappa1[i] = sample(1:C, 1, replace = TRUE, prob = prob)
  }
  return(kappa1)
}

In [6]:
data = matrix(c(1,0,1,0,0,1,1,0,0,1,1,1,1,0,0,1,1,0,0,0), nrow=5, ncol=4, byrow = TRUE)
data[1,]

[1] 1 0 1 0

In [7]:
# test data
data = matrix(c(1,0,1,0,0,1,1,0,0,1,1,1,1,0,0,1,1,0,0,0), nrow=5, ncol=4, byrow = TRUE)
theta = sample_theta(data, theta0, kappa0, C)
C = 2
pi0 = c(0.5, 0.5)
#sample_kappa(data, theta, C, pi0)

## 4. Sample $\pi^{(t)} | \kappa^{(t)}$

In [8]:
install.packages('DirichletReg')

also installing the dependencies 'miscTools', 'sandwich', 'Formula', 'maxLik'




  There are binary versions available but the source versions are later:
             binary source needs_compilation
sandwich      3.0-0  3.0-1             FALSE
maxLik        1.4-8  1.5-2             FALSE
DirichletReg  0.7-0  0.7-1              TRUE

  Binaries will be installed
package 'miscTools' successfully unpacked and MD5 sums checked
package 'Formula' successfully unpacked and MD5 sums checked
package 'DirichletReg' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Scott Turro\AppData\Local\Temp\Rtmp27TyFS\downloaded_packages


installing the source packages 'sandwich', 'maxLik'



In [9]:
require('DirichletReg')
rdirichlet(1, c(5,5,10))

Loading required package: DirichletReg
Warning message:
"package 'DirichletReg' was built under R version 3.6.3"Loading required package: Formula
Warning message:
"package 'Formula' was built under R version 3.6.3"

0.4640756,0.09550976,0.4404146


In [10]:
# Desc:
#   Samples the class probabilities using a dirichlet distrobution
# Input:
#   alphas: length C vector of alphas
# Output:
#   pi: class probabilities vector of length C
sample_pi = function(alphas) {
  # calculate alpha estimators?
  # can be equal to 1
  pi = rdirichlet(1, alphas)
  return(pi)
}

In [11]:
# Desc:
#   Samples the class probabilities using a dirichlet distrobution
# Input:
#   alpha0: length C vector of alphas
#   kappa0: length C vector of alphas
#   C: Number categories
# Output:
#   pi1: class probabilities vector of length C 
sample_pi = function(data, alpha0, kappa0, C) {
  pi1 = rep(NA, C)
  for (c in 1:C) {
    data_c = data[which(kappa0 == c),]
    n_c = dim(data_c)[1]
    # Handle the case where data_c is a vector
    if(is.null(n_c)){
      n_c = length(data_c)
    }
    alpha1 = alpha0[c] + n_c
    pi1[c] = rgamma(1,alpha1)
  }
  pi1 = pi1 / sum(pi1)
  return(pi1)
}

In [12]:
# test data
data = matrix(c(1,0,1,0,0,1,1,0,0,1,1,1,1,0,0,1,1,0,0,0), nrow=5, ncol=4, byrow = TRUE)
theta = sample_theta(data, theta0, kappa0, C)
C = 2
pi0 = c(0.5, 0.5)
alpha0 = c(1,1)
kappa0 = sample_kappa(data, theta, C, pi0)

In [13]:
sample_pi(data, alpha0, kappa0, C)

[1] 0.7374416 0.2625584

## Top Level Function

In [14]:
# Desc:
#   Main function to run the gibbs sampling algorithm 
#   and cluster the data
# Input: 
#   data: a N * P matrix
#   C: the number of category.
#   n: max number of iterations
#   b: number of burn ins
# Output:
#   list(prob, cate, theta)
#   prob: Samples for pi with dim (n-b, C)
#   cate: Samples for kappa with dim (n-b, N)
#   theta: Samples for theta with dim (n-b, c, P)

mix_binom = function(data, C, n, b=2) {
  init_value = gibbs_init(data, C)
  curr_kappa = init_value$kappa0
  beta0 = init_value$beta0
  alpha0 = init_value$alpha0
  curr_pi = init_value$pi0
  theta_l = array(rep(NA, (n-b)*C*dim(data)[2]),c((n-b),C,dim(data)[2]))
  kappa_l = matrix(rep(NA, (n-b)*dim(data)[1]),nrow = (n-b), ncol = dim(data)[1])
  pi_l = matrix(rep(NA, (n-b)*C), nrow = (n-b), ncol = C)
  for (i in 1:n) {
    curr_theta = sample_theta(data, beta0, curr_kappa, C)
    curr_kappa = sample_kappa(data, curr_theta, C, curr_pi)
    curr_pi = sample_pi(data, alpha0, curr_kappa, C)
    if (i > b) {
      theta_l[i-b,,] = curr_theta
      kappa_l[i-b,] = curr_kappa
      pi_l[i-b,] = curr_pi
    }
  }
  return(list(prob = pi_l, cate = kappa_l, theta = theta_l))
}

In [15]:
# Test with input as a matrix
test_output = mix_binom(data,2,100)
cat("dimensions of outputs",end="\n")
cat("$prob -",dim(test_output$prob),end="\n")
cat("$cat -",dim(test_output$cat),end="\n")
cat("$theta -",dim(test_output$theta),end="\n")

dimensions of outputs 
$prob - 98 2 
$cat - 98 5 
$theta - 98 2 4 


In [16]:
# Test with input as a dataframe
test_output = mix_binom(data.frame(data),2,100)
cat("dimensions of outputs",end="\n")
cat("$prob -",dim(test_output$prob),end="\n")
cat("$cat -",dim(test_output$cat),end="\n")
cat("$theta -",dim(test_output$theta),end="\n")

dimensions of outputs 
$prob - 98 2 
$cat - 98 5 
$theta - 98 2 4 


In [17]:
init_value = gibbs_init(data, C)
curr_kappa = init_value$kappa0
beta0 = init_value$beta0
alpha0 = init_value$alpha0
curr_pi = init_value$pi0

In [18]:
length(data)
#colSums(data)

[1] 20

In [19]:
# 500 obs, 10-20 questions

# Testing on NPI data

In [20]:
#install.package(devtools) or download data and put it in googledrive thingy
#library(edmdata)

install.packages("devtools")
devtools::install_github("tmsalab/edmdata")
data("items_narcissistic_personality_inventory", package = "edmdata")
head(items_narcissistic_personality_inventory)

install.packages("poLCA")
library('poLCA')

also installing the dependencies 'credentials', 'openssl', 'zip', 'gitcreds', 'ini', 'fastmap', 'diffobj', 'desc', 'gert', 'gh', 'rappdirs', 'processx', 'cachem', 'xopen', 'brew', 'commonmark', 'purrr', 'cpp11', 'brio', 'ps', 'waldo', 'usethis', 'callr', 'cli', 'ellipsis', 'fs', 'httr', 'lifecycle', 'memoise', 'pkgbuild', 'pkgload', 'rcmdcheck', 'rlang', 'roxygen2', 'rstudioapi', 'rversions', 'sessioninfo', 'testthat', 'withr'




  There are binary versions available but the source versions are later:
            binary source needs_compilation
credentials  1.3.0  1.3.1             FALSE
openssl      1.4.4  1.4.5              TRUE
zip          2.1.1  2.2.0              TRUE
diffobj      0.3.4  0.3.5              TRUE
desc         1.3.0  1.4.0             FALSE
gert         1.3.0  1.4.1              TRUE
cachem       1.0.4  1.0.6              TRUE
cpp11        0.2.7  0.4.0             FALSE
waldo        0.2.5  0.3.1             FALSE
usethis      2.0.1  2.1.3             FALSE
cli          2.5.0  3.1.0              TRUE
lifecycle    1.0.0  1.0.1             FALSE
pkgload      1.2.1  1.2.3             FALSE
rcmdcheck    1.3.3  1.4.0             FALSE
rlang       0.4.11 0.4.12              TRUE
roxygen2     7.1.1  7.1.2              TRUE
rversions    2.0.2  2.1.1             FALSE
testthat     3.0.2  3.1.0              TRUE
devtools     2.4.1  2.4.2             FALSE

  Binaries will be installed
package 'openssl

installing the source packages 'credentials', 'desc', 'cpp11', 'waldo', 'usethis', 'lifecycle', 'pkgload', 'rcmdcheck', 'rversions', 'devtools'

Warning message in install.packages("devtools"):
"installation of package 'rcmdcheck' had non-zero exit status"Warning message in install.packages("devtools"):
"installation of package 'devtools' had non-zero exit status"

ERROR: Error in loadNamespace(name): there is no package called 'devtools'


In [ ]:
start_time <- Sys.time()
mix_binom(items_narcissistic_personality_inventory, 8, 300, 100)
cat("Total time:",Sys.time()-start_time,"(mins)")

In [ ]:
#data is 40 questions, 25 participants, 0 or 1 (25x40) 

#choosing best number of groups
data= items_narcissistic_personality_inventory
data=data+1

library('poLCA')

varnames<-paste0('Q',1:40)
Qs <- paste(varnames, collapse = ",")
f <- as.formula(paste("cbind(", Qs, ")~1 "))

X= as.data.frame(data[,varnames])
X=X[complete.cases(X),]
out2<-poLCA(f, X, nclass = 2)

#now, how do we know the best number of classes to split on?
#answer : use bics (also ask proff if there is better ways)

nclasses=2:8

model_bics= numeric(length(nclasses))
for(i in nclasses){
  outtemp=poLCA(f,X,nclass=i,maxiter = 2000)
  assign(paste0('out',i),outtemp)
  model_bics[i-1]=outtemp$bic
}

names(model_bics)=nclasses
model_bics

#best fitting is 8 classes!! :0
#we can now try to find what each class means. Personalities? jobs? if they like stats?

C=8




# Testing with Mr. Turro's Data

## Grabbing Data

In [ ]:
#doing same thing but with Mr. Turro's data

# data has has answers for 20 questions from 1000 subjects
data = read.csv("https://raw.githubusercontent.com/smturro2/URES-Project-2/master/Data/test/test_data.csv",row.names = 1)
data_matrix = as.matrix(data)
print(dim(data_matrix))

# True K has length 1000. The classes of each subject
true_k = read.csv("https://raw.githubusercontent.com/smturro2/URES-Project-2/master/Data/test/test_k_vector.csv",row.names = 1)
true_k_matrix = as.matrix(true_k)
print(dim(true_k_matrix))

# True pi vector has length 4. The probability of being in a class
true_pi = read.csv("https://raw.githubusercontent.com/smturro2/URES-Project-2/master/Data/test/test_pi_vector.csv",row.names = 1)
true_pi_matrix = as.matrix(true_pi)
print(dim(true_pi_matrix))

# True theta matrix is shape 4x20. How each class does on each question
true_theta = read.csv("https://raw.githubusercontent.com/smturro2/URES-Project-2/master/Data/test/test_theta_matrix.csv",row.names = 1)
true_theta_matrix = as.matrix(true_theta)
print(dim(true_theta_matrix))

## Running algo

In [ ]:
# convert data into matrix 
data = read.csv("https://raw.githubusercontent.com/smturro2/URES-Project-2/master/Data/test/test_data.csv")
colnames(data) = NULL
data = data[,-1]
data = as.matrix(data)
dim(data)

In [ ]:
# Run the algo and time how long it takes
start_time <- Sys.time()
test_output = mix_binom(data,4,500,100)
cat("Total time:",Sys.time()-start_time,"(mins)")

# Run the algo as a data frame and time how long it takes
start_time <- Sys.time()
test_output = mix_binom(data.frame(data),4,500,100)
cat("Total time:",Sys.time()-start_time,"(mins)")

In [ ]:
as.matrix(colMeans(test_output$prob))

In [ ]:
true_pi_matrix

In [ ]:
# Get average values
pi_hat = as.matrix(colMeans(test_output$prob))
k_hat = as.matrix(rowMeans(test_output$cate))
#theta_hat = as.matrix(rowMeans(as.matrix(test_output$theta), dims = 2)) # todo

cat("dimensions of avg parameters",end="\n")
cat("pi_hat -",dim(pi_hat),end="\n")
cat("k_hat -",dim(k_hat),end="\n")
#cat("theta_hat -",dim(theta_hat),end="\n")

In [ ]:
true_k_matrix

In [ ]:
k_hat

In [ ]:
install.packages("devtools")